In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
from splinter import Browser
import tweepy
from textblob import TextBlob, Word, Blobber
import time
import pandas as pd
import urllib3,certifi

# Twitter API Keys
from config import *

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
#get headline and text from NASA website
url = "https://mars.nasa.gov/news/"
response = requests.get(url, 'lxml')

# Create a Beautiful Soup object
soup = bs(response.text, 'html.parser')

#Scrape the headline and text for the most recent article. Strip the escape characters.
news_title = soup.find('div', class_="content_title").find("a").text.strip('\t\r\n')
news_p = soup.find('div', class_="rollover_description").text.strip('\t\r\n')

#print(news_title)
#print(news_p)

Dust Storms Linked to Gas Escape from Mars Atmosphere
If Mars has a global dust storm in 2018, observations could aid understanding of its effects.


In [3]:
#Get Mars weather from twitter
user='@marswxreport'
tweet = api.user_timeline(user, count = 1)
mars_weather = tweet[0]['text']
#print(mars_weather)

Sol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28


In [4]:
#Get Featured Mars image
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
browser.quit()

output = soup.find_all('article')
featured_image_url="https://www.jpl.nasa.gov"+output[0].find("a")["data-fancybox-href"]

In [5]:
%%capture



'''#Scrape the headline and text for the most recent article. Strip the escape characters.
news_title = soup.find('div', class_="content_title").find("a").text.strip('\t\r\n')
news_p = soup.find('div', class_="rollover_description").text.strip('\t\r\n')

print(news_title)
print(news_p)'''

In [13]:
#scrape the table containing facts about the planet including Diameter, Mass, etc.

#Get Mars facts
url = "https://space-facts.com/mars/"
response = requests.get(url, 'lxml')

# Create a Beautiful Soup object
soup = bs(response.text, 'html.parser')

table=soup.find('table')

rows = table.findAll('tr')

for tr in rows:
    cols = tr.findAll('td')
    #print(cols)

#convert to pandas dataframe
headings=[]
contents=[]

for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        #new_table.iat[row_marker,column_marker] = column.get_text()
        if column_marker==0:
            output=column.get_text().strip()
            headings.append(output)
            #print(headings)
        else:
            output=column.get_text().strip()
            contents.append(output)
            #print(contents)
        column_marker += 1
    
df=pd.DataFrame(contents,headings)


#convert to HTML table
d = {'col1': headings, 'col2': contents}
df = pd.DataFrame(data=contents, index=headings)
#df
df_html_output = df.to_html(header=False)
#df_html_output

#This includes a bunch of \n at the top which is lame but idk how to make it go away

,0
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [36]:
#Get current timestamp

import datetime

now=datetime.datetime.now()

timestamp=datetime.datetime.strftime(now, '%Y-%m-%d %H:%S%p')

'2018-01-27 09:16AM'

In [38]:
def Scrape():
    output = {
        "timestamp":timestamp,
        "news_title":news_title,
        "news_p":news_p,
        "mars_weather":mars_weather,
        "featured_image_url":featured_image_url,
        "df_html_output":df_html_output
            }
    return output

In [19]:
#Scrape(news_title,news_p,mars_weather,featured_image_url,df_html_output)

{'df_html_output': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA02570_ip.jpg',
 'mars_weather': 'Sol 1945 (Jan 25,

In [22]:
timestamp

datetime.datetime(2018, 1, 27, 9, 47, 13, 913257)

In [37]:
from pymongo import Connection

ImportError: cannot import name 'Connection'